# Задача о принадлежности точки полигону

In [1]:
# Импортируем библиотеки
from time import time
import matplotlib.path as mpltPath
import pandas as pd
import numpy as np

In [2]:
# Получаем датасет с координатами пользователя на плоскости (можно считать условно х и у)
user_coordinates = pd.read_csv("user_coordinates.csv")

In [3]:
# Просматриваем первые 10 строк массива
user_coordinates.head(10)

,user_id,loc_lat,loc_lon
0,1,0.5,0.5
1,2,3.0,3.0
2,3,5.0,5.0


In [4]:
# Получаем датасет с координатами полигона на плоскости. Полигон определяется 4 вершинами. 
# Указание вершин начинается с левого нижнего угла. Движение слева направо, снизу вверх (т.е. против часовой стрелки)
place_zone_coordinates = pd.read_csv("place_zone_coordinates.csv")

In [5]:
place_zone_coordinates.head(10)

,place_id,loc_lat,loc_lon,point_number
0,1,1,1,0
1,1,4,1,1
2,1,4,4,2
3,1,1,4,3
4,2,0,0,0
5,2,6,0,1
6,2,6,6,2
7,2,0,6,3


In [6]:
# Перестраиваем исходный массив информации, чтобы в дальнейшем упростить построение списка с координатами полигона
place_zone_coordinates = place_zone_coordinates.pivot(index="place_id", columns="point_number", values = ["loc_lat", "loc_lon"])
place_zone_coordinates.columns = ["x1","x2","x3","x4","y1","y2","y3","y4" ]
place_zone_coordinates = place_zone_coordinates.reset_index()
place_zone_coordinates.head(10)

,place_id,x1,x2,x3,x4,y1,y2,y3,y4
0,1,1,4,4,1,1,1,4,4
1,2,0,6,6,0,0,0,6,6


In [7]:
# Создаем списоки с уникальными id пользователей и полигонов
place_id_array = place_zone_coordinates["place_id"].unique()
user_id_array = user_coordinates["user_id"].unique()

In [8]:
# Запускаем цикл проверки. Результаты подсчета пишем в словарь. По ходу решения выводим print для самопроверки. 
user_id_number_of_places_available ={}

for user_id in user_id_array:
    user_coordinates_filter = user_coordinates[user_coordinates["user_id"] == user_id]
    point_user = [[round( user_coordinates_filter ["loc_lat"].values[0], 6),
                  round( user_coordinates_filter ["loc_lon"].values[0], 6)]]
    print(point_user)
    count_places_available = 0
    for place_id in place_id_array:
        place_zone_coordinates_filter = place_zone_coordinates[(place_zone_coordinates["place_id"] == place_id)]
        polygon = [
            (
            round(place_zone_coordinates_filter ["x1"].values[0],6), 
            round(place_zone_coordinates_filter["y1"].values[0],6)
            ),
            (
            round(place_zone_coordinates_filter["x2"].values[0],6), 
            round(place_zone_coordinates_filter["y2"].values[0],6)
            ),
            (
            round(place_zone_coordinates_filter["x3"].values[0],6), 
             round(place_zone_coordinates_filter["y3"].values[0],6)
            ),
            (
             round(place_zone_coordinates_filter["x4"].values[0],6), 
             round(place_zone_coordinates_filter["y4"].values[0],6)
            )
        ]
        print(polygon)
        path = mpltPath.Path(polygon)
        inside = path.contains_points(point_user)
        print(inside)
        if inside:
            count_places_available += 1 
    user_id_number_of_places_available[user_id] = count_places_available
print(user_id_number_of_places_available)

[[0.5, 0.5]]
[(1, 1), (4, 1), (4, 4), (1, 4)]
[False]
[(0, 0), (6, 0), (6, 6), (0, 6)]
[ True]
[[3.0, 3.0]]
[(1, 1), (4, 1), (4, 4), (1, 4)]
[ True]
[(0, 0), (6, 0), (6, 6), (0, 6)]
[ True]
[[5.0, 5.0]]
[(1, 1), (4, 1), (4, 4), (1, 4)]
[False]
[(0, 0), (6, 0), (6, 6), (0, 6)]
[ True]
{1: 1, 2: 2, 3: 1}


In [9]:
# Трансформируем словарь в датафрейм
df = pd.DataFrame({'user_id' : [i for i in user_id_number_of_places_available.keys()], 
                   'number_of_places_available' : [ii for ii in user_id_number_of_places_available.values()]})
print(df)

   user_id  number_of_places_available
0        1                           1
1        2                           2
2        3                           1
